# HW2

Jaee Oh / Tejas Phanse / Mandadi Ayush Reddy

## Part A: DTM

In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet') 
print("Libraries loaded successfully.")

Libraries loaded successfully.


[nltk_data] Downloading package stopwords to /home/jaee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jaee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


- Text Prerpcessing

In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# List of common words to be removed from full_text_clean column
common_word_list = {'followers', 'instagram', 'facebook', 'like', 'messages', 'support',
                    'sitemap', 'free', 'faq', 'twitter', 'search', 'subscribe', 'youtube', 
                    'feedback', 'linkedin', 'demo', 'menu', 'following', 'unsubscribe', 
                    'snapchat', 'member', 'tiktok', 'whatsapp', 
                    'profile', 'share', 'reddit', 'download', 'settings', 'notifications', 
                    'home', 'qa', 'people'}
stop_words = stop_words.union(common_word_list) # Add common words to NLTK stop words

# Common phrases to remove
phrase_list = ["find out more", "join now","terms & conditions", "terms and conditions","all rights reserved","free trial", 
             "click here", "about us", "contact us","terms of service", "privacy policy", "help center", "our story","our team",
             "read more", "learn more", "get started", "download now", "sign up", "log in"]

def preprocess(text):
    # Remove \n, \r, \t
    text = re.sub(r'[\n\r\t]+', ' ', text)

    # Remove HTML tags like <body>, but first remove script/style content
    soup = BeautifulSoup(text, 'html.parser')
    for tag in soup(['script', 'style', 'noscript']):
        tag.decompose()
    text = soup.get_text(separator=' ')
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'(?:https?://|ftp://|www\.)\S+|mailto:\S+|tel:\S+', '', text)
    
    # Remove emails
    text = re.sub(r'\b[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b', '', text)
    
    # Remove phrases
    for phrase in phrase_list:
        text = text.replace(phrase, ' ')

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]', ' ', text)
    
    # Keep only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize and filter with stop words and lemmatization
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(t) for t in tokens if len(t) > 2 and t not in stop_words]
    
    return ' '.join(tokens)

In [ ]:
# Load documents
documents = pd.read_csv('../data/dataset_with_assignments.csv')
# Apply preprocessing
documents['text_clean'] = documents['full_text'].apply(preprocess)

# Basic CountVectorizer
count_vec = CountVectorizer()
count_matrix = count_vec.fit_transform(documents['text_clean'])  # Fixed

print(f"Matrix shape: {count_matrix.shape}")
print(f"  - {count_matrix.shape[0]} documents")
print(f"  - {count_matrix.shape[1]} unique terms in vocabulary")

# Be careful with large vocabularies - this could print thousands of words
vocab = count_vec.get_feature_names_out()
print(f"\nFirst 20 terms: {vocab[:20]}")
print(f"Last 20 terms: {vocab[-20:]}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/dataset_with_assignment.csv'

- TF-IDF Vectorizer

In [11]:
tfidf_vec = TfidfVectorizer(
    max_df=0.85,      # Ignore terms in >85%
    min_df=0.15,         # Ignore terms in <15%
    ngram_range=(1,2), # Unigrams and bigrams
    max_features=10000  # Max vocabulary size
)
tfidf_matrix = tfidf_vec.fit_transform(documents['text_clean'])

print(tfidf_matrix.shape)

(25, 161)
